# Joe Stanley
### ECE522 - EXAM1

In [3]:
# Import Necessary Libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import electricpy as ep
from electricpy.constants import *

## Problem 1:
Given the parameters described below, find: $\text{slip}_{\text{rated}}$, $\omega_{r-\text{rated}}$, and $\Lambda_{dr_\text{rated}}$. Rated is defined to mean that the operating conditions are such that: $\left|V_{dqs}\right|=1.0\text{pu}$, $T_{em}=1.0\text{pu}$, and $\omega_{es}=1.0\text{pu}$

For this, we know:

$\\
V_{dqs}=r_sI_{dqs}+j\omega_{es}\left(L_sI_{dqs}+\frac{3}{2}L_{sr}I_{dqr}\right)\\
0=r_rI_{dqr}+j(\omega_{es}-\omega_r)\Lambda_{dqr}\\
\Lambda_{dqr}=\frac{3}{2}L_{sr}I_{dqs}+L_{r}I_{dqr}\\
T_{em}=\frac{3p}{4}\frac{\frac{3}{2}L_{sr}}{L_r}\operatorname{Im}\left(\overline{\Lambda_{dqr}}I_{dqs}\right)\\
$

Additionally, we will need to use a few additional equations to solve this system.

$\\
L_s=L_{LS}+L_m\\
L_r=L_{Lr}+L_m\\
L_{sr} = \frac{2}{3}\cdot(L_r - L_{Lr})\cdot a_t
$

As a final note, we will make the following assumptions:

$p=4\qquad a_t=2$

Using an iterative solver (shown in code below), we can now find the terms as:

{{P1_latex}}

From simple manipulation and calculation, we can find the desired results to equal:

- $\text{slip}_{\text{rated}}=${{s_rated}}
- $\omega_{r_\text{rated}}=${{w_rated}}
- $\Lambda_{\text{dr}_\text{rated}}=${{lamdr_rated}}

In [26]:
# Define Provided Machine Parameters
rs = 0.03 #pu
LLs = 0.1 #pu
Lm = 2.0 #pu
LLr = 0.1 #pu
rr = 0.03 #pu

# Define Rated Criteria
VdqsMag = 1
Tem = 1
wes = 1

# Assumptions
p = 4
at = 2

# Calculate Additional Inductance Terms
Ls = LLs + Lm
Lr = LLr + Lm
Lmr = 2/3 * (Lr - LLr)
Lsr = Lmr * at

# Define Equations Function as Solver
def equations(val):
    Idr,Iqr,Ids,Iqs,LAMdr,LAMqr,LAMds,LAMqs,wr = val
    A = (rs*Ids - wes*LAMqs) - VdqsMag
    B = rs*Iqs - wes*LAMds
    C = rr*Idr - (wes-wr)*LAMqr
    D = rr*Iqr + (wes-wr)*LAMdr
    E = (Ls*Ids + 3/2*Lsr*Idr) - LAMds
    F = (Ls*Iqs + 3/2*Lsr*Iqr) - LAMqs
    G = (3/2*Lsr*Ids+Lr*Idr) - LAMdr
    H = (3/2*Lsr*Iqs+Lr*Iqr) - LAMqr
    I = (3*p/4*(3/2*Lsr)/Lr*(LAMdr*Iqs-LAMqr*Ids)) - Tem
    return(A,B,C,D,E,F,G,H,I)

# Define Initial Guesses
Idr0 = -1
Iqr0 = -1
Ids0 = 1
Iqs0 = -1
LAMdr0 = 3/2*Lsr*Ids0 + Lr*Idr0
LAMqr0 = 3/2*Lsr*Iqs0 + Lr*Iqr0
LAMds0 = Ls*Ids0 + 3/2*Lsr*Idr0
LAMqs0 = Ls*Iqs0 + 3/2*Lsr*Iqr0
wr = 360

# Use Iterative Solver to Find Results
Idr,Iqr,Ids,Iqs,LAMdr,LAMqr,LAMds,LAMqs,wr = fsolve(equations,(
    Idr0,Iqr0,Ids0,Iqs0,LAMdr0,LAMqr0,LAMds0,LAMqs0,wr))

# Compose Vectors
Idqr = Idr + 1j*Iqr
Idqs = Ids + 1j*Iqs
LAMdqr = LAMdr+1j*LAMqr
LAMdqs = LAMds+1j*LAMqs

# Generate into Latex Vector
P1_latex = r"$$\begin{bmatrix}I_{\text{dqr}}\\ I_{\text{dqs}}\\ \Lambda_{\text{dqr}}\\ \Lambda_{\text{dqr}}\end{bmatrix}="
P1_latex += ep.clatex([Idqr,Idqs,LAMdqr,LAMdqs],predollar=False,double=True)

# Calculate Desired Terms
s_rated = round((wes-wr)/wes,5)
w_rated = round(wr,5)
lamdr_rated = round(abs(LAMdqr),5)